In [2]:
import os
import base64
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import streamlit as st
from IPython.display import Markdown

load_dotenv(find_dotenv())

True

In [4]:
import openai

# Initialize the OpenAI client
client = openai.OpenAI()

# Fetch the list of models
response = client.models.list()

# Extract models from the response
models = response.data  # Use .data to access the list of models

# Print only the model IDs
print("Available Model Names:")
for model in models:
    print(model.id)


Available Model Names:
dall-e-3
dall-e-2
tts-1-hd-1106
tts-1-hd
text-embedding-3-large
babbage-002
gpt-4o-mini
gpt-4o-mini-2024-07-18
text-embedding-3-small
tts-1
gpt-3.5-turbo
whisper-1
text-embedding-ada-002
gpt-3.5-turbo-16k
davinci-002
tts-1-1106
gpt-3.5-turbo-0125
gpt-3.5-turbo-1106
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct


In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


In [7]:
#@title Encode the Image 
def encode_image(image_path: str) -> str:
    with open(image_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_image

In [8]:
#@title Analyze the Image
def analyze_image(image_data: str) -> str:
    prompt_instructions = """
                          You are an intelligent assistant whose task is to analyze the food order and calculate the carbon footprint.

                         **Instructions**:
                            1. **Identify the Food Items**:
                            - Determine if each food item is vegetarian or non-vegetarian.
                            - List each food item with its classification (veg/non-veg).

                            2. **Calculate Carbon Emissions**:
                            - Use India-specific carbon emission factors to calculate the carbon emissions for each food item.
                            - Provide the carbon emission factors used for each food item.

                            3. **Provide a Breakdown**:
                            - Detail the carbon emissions for each food item separately.
                            - Summarize the total carbon emissions for the entire order.

                            4. **Summarize the Results in brief**:
                            - Include a brief summary of the order.
                            - Mention the type (veg/non-veg).
                            - State the total carbon emissions.
                            - Provide a breakdown of the emissions.
                            """
    
    client = OpenAI()
    MODEL = "gpt-4o"

    response = client.chat.completions.create(
        model = MODEL,
        messages = [
            {"role" : "system", "content" : prompt_instructions},
            {"role" : "user", "content" : [
                    {"tyep" : "image_url", "image_url" : {"url" : f"data:image/png;base64{image_data}"}} # Replace with the actual image data
            ]}
        ],
        temperature = 0.7
    )
    return response.choices[0].message.content

    
                             

In [9]:
st.set_page_config(
    layout = "wide"
)

st.sidebar.title("Food Order Carbon Foodprint Calculator")
uploaded_file = st.sidebar.file_uploader("Upload an Image of Your food order", type = ["jpg","png","jpeg"])

if st.sidebar.button("Calculate"):
    if uploaded_file is not None:
        with open("uploaded_image.png", "wb") as f:
            f.write(uploaded_file.getbuffer())

            # Encode the Image
            base_64_image = encode_image("uploaded_image.png")

            # Analyze the Image
            result = analyze_image(base_64_image)

            # Display
            col1, col2 = st.columns(2)

            with col1:
                st.image("uploaded_image.png", caption = "Uploaded Food Order Recipet", use_column_width = True)

            with col2:
                st.markdown(result)

    else:
        st.sidebar.error("Please upload your image to proceed.")


2024-08-12 15:28:52.307 
  command:

    streamlit run /opt/homebrew/Caskroom/miniconda/base/envs/carbon/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
